# TCS Stock Data – One-Click Colab Run

This notebook is designed for **Run all** in Google Colab. It will:
1. Clone the GitHub repository
2. Install dependencies
3. Download latest TCS stock data from Yahoo Finance
4. Run quick EDA plots
5. Train baseline ML models (Linear Regression + Random Forest)

> Recommended: Runtime -> Run all

In [ ]:
# STEP 1: Clone repo (if needed) and move into project folder
import os
import sys

REPO_URL = "https://github.com/ManishPatil2005/tcs-stock-data-live-latest.git"
REPO_DIR = "/content/tcs-stock-data-live-latest"

if not os.path.exists(REPO_DIR):
    !git clone {REPO_URL}

%cd /content/tcs-stock-data-live-latest
print("Current directory:", os.getcwd())

In [ ]:
# STEP 2: Install project dependencies
!pip -q install -r requirements.txt

In [ ]:
# STEP 3: Download latest TCS data (online)
!python src/download_latest_tcs_data.py --period 5y --interval 1d

In [ ]:
# STEP 4: Load data and show quick preview
import pandas as pd

from src.data_loader import load_tcs_data

df = load_tcs_data()
print("Shape:", df.shape)
display(df.head())
display(df.describe())

In [ ]:
# STEP 5: Quick EDA plots
import matplotlib.pyplot as plt

from src.eda import plot_close_price, plot_volume, plot_correlation_heatmap, plot_moving_averages

plot_close_price(df)
plot_volume(df)
plot_correlation_heatmap(df)
plot_moving_averages(df, short_window=5, long_window=30)

In [ ]:
# STEP 6: Build features + run baseline ML models
from src.features import build_features
from src.models import run_linear_regression, run_random_forest

feature_df = build_features(df)
print("Feature shape:", feature_df.shape)

lr_model = run_linear_regression(feature_df)
rf_model = run_random_forest(feature_df, n_estimators=100)

print("Done: Baseline models executed successfully.")

## Final Outcome (Main Result)
This section gives a clear, presentation-ready summary based on moving-average crossover logic:
- **Current Bias**: BUY or SELL
- **Latest Event**: NEW BUY / NEW SELL / HOLD
- **Actionable Final Decision**: Final recommendation text for demo/interview use

In [ ]:
# FINAL DECISION LOGIC (clear internship outcome)
short_window = 5
long_window = 30

signal_df = df.copy()
signal_df['MA_short'] = signal_df['Close'].rolling(short_window).mean()
signal_df['MA_long'] = signal_df['Close'].rolling(long_window).mean()
signal_df['Signal'] = (signal_df['MA_short'] > signal_df['MA_long']).astype(int)
signal_df['Position'] = signal_df['Signal'].diff()

latest = signal_df.iloc[-1]
latest_date = pd.to_datetime(latest['Date']).date()

current_bias = 'BUY' if int(latest['Signal']) == 1 else 'SELL'
position_value = latest['Position']
if pd.isna(position_value) or position_value == 0:
    latest_event = 'HOLD (no new crossover today)'
elif position_value == 1:
    latest_event = 'NEW BUY crossover today'
else:
    latest_event = 'NEW SELL crossover today'

print('================ FINAL OUTCOME ================')
print(f'Date           : {latest_date}')
print(f'Current Bias   : {current_bias}')
print(f'Latest Event   : {latest_event}')
print(f'Latest Close   : {latest["Close"]:.2f}')
print(f'MA({short_window})        : {latest["MA_short"]:.2f}')
print(f'MA({long_window})       : {latest["MA_long"]:.2f}')
print('================================================')

if current_bias == 'BUY' and 'HOLD' in latest_event:
    print('Final Decision: BUY (trend still bullish, no fresh crossover today)')
elif current_bias == 'SELL' and 'HOLD' in latest_event:
    print('Final Decision: SELL (trend still bearish, no fresh crossover today)')
elif 'NEW BUY' in latest_event:
    print('Final Decision: BUY (fresh bullish crossover)')
else:
    print('Final Decision: SELL (fresh bearish crossover)')

# Optional quick display table for report screenshots
summary_df = pd.DataFrame([
    {
        'Date': latest_date,
        'Current_Bias': current_bias,
        'Latest_Event': latest_event,
        'Latest_Close': round(float(latest['Close']), 2),
        f'MA_{short_window}': round(float(latest['MA_short']), 2),
        f'MA_{long_window}': round(float(latest['MA_long']), 2),
    }
])

display(summary_df)

## Optional
If you want, you can also run LSTM from `src.models.run_lstm(df)` in a separate cell.
It may take longer and needs TensorFlow runtime support.